# Basic EM workflow 1 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow to match two tables using *py_entitymatching*. Our goal is to match restaurants from Fodors and Zagat sites. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/fodors.csv'
path_B = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/zagats.csv'

In [4]:
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

No handlers could be found for logger "py_entitymatching.io.parsers"


In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 533
Number of tuples in B: 331
Number of tuples in A X B (i.e the cartesian product): 176423


In [6]:
A.head()

,id,name,addr,city,phone,type
0,534,arnie mortons of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american
1,535,arts delicatessen,12224 ventura blvd.,studio city,818/762-1221,american
2,536,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian
3,537,cafe bizou,14016 ventura blvd.,sherman oaks,818/788-3536,french
4,538,campanile,624 s. la brea ave.,los angeles,213/938-1447,american


In [7]:
B.head()

,id,name,addr,city,phone,type
0,1,apple pan the,10801 w. pico blvd.,west la,310-475-3585,american
1,2,asahi ramen,2027 sawtelle blvd.,west la,310-479-2231,noodle shops
2,3,baja fresh,3345 kimber dr.,westlake village,805-498-4049,mexican
3,4,belvedere the,9882 little santa monica blvd.,beverly hills,310-788-2306,pacific new wave
4,5,benitas frites,1433 third st. promenade,santa monica,310-458-2889,fast food


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with no overlap between the names will not match. So we decide the apply blocking over names:

In [9]:
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'name', 'name', 
                    l_output_attrs=['name', 'addr', 'city', 'phone'], 
                    r_output_attrs=['name', 'addr', 'city', 'phone'],
                    overlap_size=1, show_progress=False)

In [10]:
C.head()

,_id,ltable_id,rtable_id,ltable_name,ltable_addr,ltable_city,ltable_phone,rtable_name,rtable_addr,rtable_city,rtable_phone
0,0,927,1,dantes down the hatch buckhead,3380 peachtree rd.,atlanta,404/266-1600,apple pan the,10801 w. pico blvd.,west la,310-475-3585
1,1,695,1,rjs the rib joint,252 n. beverly dr.,los angeles,310/274-7427,apple pan the,10801 w. pico blvd.,west la,310-475-3585
2,2,1054,1,the heights,3235 sacramento st.,san francisco,415/474-8890,apple pan the,10801 w. pico blvd.,west la,310-475-3585
3,3,983,1,the bacchanal,3570 las vegas blvd. s,las vegas,702/731-7525,apple pan the,10801 w. pico blvd.,west la,310-475-3585
4,4,894,1,the coffee pot,350 9th ave. at 49th st.,new york,212/265-3566,apple pan the,10801 w. pico blvd.,west la,310-475-3585


# Match tuple pairs in candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following four steps:

1. Sampling and labeling the candidate set
2. Train matcher using labeled data
3. Predict the matches in the candidate set using trained matcher

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [11]:
# Sample  candidate set
S = em.sample_table(C, 450)

Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [12]:
# Label S
G = em.label_table(S, 'gold')

/anaconda2/lib/python2.7/site-packages/py_entitymatching/gui/table_gui.py:94: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  table.set_value(idxv[i], cols[j], val)


For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [13]:
path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(G)

450

## Train matcher using labeled data

First, we need to create a set of features.*py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [28]:
# Generate features automatically 
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)
feature_table.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x12383ba28>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x12383b8c0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x12383eed8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x12383eb90>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x12383ef50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


Next, we convert the labeled data to feature vectors using the feature table

In [15]:
# Select the attrs. to be included in the feature vector table
attrs_from_table = ['ltable_name', 'ltable_addr', 'ltable_city', 'ltable_phone',
                    'rtable_name', 'rtable_addr', 'rtable_city', 'rtable_phone']
# Convert the labeled data to feature vectors using the feature table
H = em.extract_feature_vecs(G, 
                            feature_table=feature_table, 
                            attrs_before = attrs_from_table,
                            attrs_after='gold',
                            show_progress=False)

Then, we train the learning-based matcher using the feature vectors. For the purposes of the guide, we will use Random Forest matcher that is included in the *py_entitymatching* package.

In [16]:
# Instantiate the RF Matcher
rf = em.RFMatcher()

In [17]:
# Get the attributes to be projected while training
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_id', 'rtable_id', 'gold'])
attrs_to_be_excluded.extend(attrs_from_table)

In [18]:
# Train using feature vectors from the labeled data.
rf.fit(table=H, exclude_attrs=attrs_to_be_excluded, target_attr='gold')

## Predict the matches in the candidate set using trained matcher

Now, we use the trained matcher to predict matches in the candidate set. To do that, first we need to convert the candidate set to feature vectors.

In [19]:
# Select the attrs. to be included in the feature vector table
attrs_from_table = ['ltable_name', 'ltable_addr', 'ltable_city', 'ltable_phone',
                    'rtable_name', 'rtable_addr', 'rtable_city', 'rtable_phone']
# Convert the cancidate set to feature vectors using the feature table
L = em.extract_feature_vecs(C, feature_table=feature_table,
                             attrs_before= attrs_from_table,
                             show_progress=False, n_jobs=-1)

Next, we predict the matches in the candidate set using the trained matcher and the feature vectors.

In [20]:
# Get the attributes to be excluded while predicting 
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_id', 'rtable_id'])
attrs_to_be_excluded.extend(attrs_from_table)

In [21]:
# Predict the matches
predictions = rf.predict(table=L, exclude_attrs=attrs_to_be_excluded,                          
              append=True, target_attr='predicted', inplace=False)

In [32]:
predictions[predictions.predicted == 1].head()

,_id,ltable_id,rtable_id,ltable_name,ltable_addr,ltable_city,ltable_phone,rtable_name,rtable_addr,rtable_city,rtable_phone,predicted
798,798,892,114,teresas,103 1st ave. between 6th and 7th sts.,new york,212/228-0604,teresas,80 montague st.,queens,718-520-2910,1
1432,1432,661,191,cha cha cha,656 n. virgil ave.,los angeles,213/664-7723,cha cha chas,1805 haight st.,san francisco,415-386-5758,1
1725,1725,534,219,arnie mortons of chicago,435 s. la cienega blv.,los angeles,310/246-1501,arnie mortons of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,1
1739,1739,535,220,arts delicatessen,12224 ventura blvd.,studio city,818/762-1221,arts deli,12224 ventura blvd.,studio city,818-762-1221,1
1740,1740,536,221,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,1


Finally, project the attributes and the predictions from the predicted table.

In [31]:
# Get the attributes to be projected out
attrs_proj = []
attrs_proj.extend(['_id', 'ltable_id', 'rtable_id'])
attrs_proj.extend(attrs_from_table)
attrs_proj.append('predicted')

# Project the attributes
predictions = predictions[attrs_proj]

In [33]:
predictions[predictions.predicted == 1].head()

,_id,ltable_id,rtable_id,ltable_name,ltable_addr,ltable_city,ltable_phone,rtable_name,rtable_addr,rtable_city,rtable_phone,predicted
798,798,892,114,teresas,103 1st ave. between 6th and 7th sts.,new york,212/228-0604,teresas,80 montague st.,queens,718-520-2910,1
1432,1432,661,191,cha cha cha,656 n. virgil ave.,los angeles,213/664-7723,cha cha chas,1805 haight st.,san francisco,415-386-5758,1
1725,1725,534,219,arnie mortons of chicago,435 s. la cienega blv.,los angeles,310/246-1501,arnie mortons of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,1
1739,1739,535,220,arts delicatessen,12224 ventura blvd.,studio city,818/762-1221,arts deli,12224 ventura blvd.,studio city,818-762-1221,1
1740,1740,536,221,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,1
